# 한국투자증권 REST API

## 1. 서비스 연결(Oauth)

In [19]:
import requests # HTTP 요청을 보낼 때
import json # 특정 객체를 JSON 데이터로 만들어서 쓰기 위해
from config.user_info import *

In [20]:
# 모의투자
URL_BASE = "https://openapivts.koreainvestment.com:29443"

In [21]:
headers = {
    "content-type": "application/json"
    }
body = {
    "grant_type": "client_credentials",
    "appkey": APP_KEY,
    "appsecret": APP_SECRET
    }
PATH = "oauth2/tokenP"

URL = f"{URL_BASE}/{PATH}"
print(URL)

https://openapivts.koreainvestment.com:29443/oauth2/tokenP


In [22]:
res = requests.post(URL, headers=headers, data=json.dumps(body))

In [23]:
# 보안인증키 : access_token
ACCESS_TOKEN = res.json()["access_token"]

## 2. 주식현재가 조회
### 현재가 : FHKST01010100
### 일자별 : FHKST01010400

In [24]:
# 현재가 조회
PATH = "uapi/domestic-stock/v1/quotations/inquire-price"
URL = f"{URL_BASE}/{PATH}"
print(URL)

headers = {
    "content-type": "application/json",
    "authorization": f"Bearer {ACCESS_TOKEN}",
    "appkey": APP_KEY,
    "appsecret": APP_SECRET,
    "tr_id": "FHKST01010100"
    }

params = {
    "fid_cond_mrkt_div_code": "J",
    "fid_input_iscd": "005930"
}

https://openapivts.koreainvestment.com:29443/uapi/domestic-stock/v1/quotations/inquire-price


In [25]:
res = requests.get(URL, headers=headers, params=params)

In [26]:
res.json()["output"]["stck_prpr"]

'59000'

In [27]:
# 일자별 조회
PATH = "uapi/domestic-stock/v1/quotations/inquire-daily-price"
URL = f"{URL_BASE}/{PATH}"
headers["tr_id"] = "FHKST01010400"

params = {
    "fid_cond_mrkt_div_code": "J",
    "fid_input_iscd": "005930",
    "fid_org_adj_prc": "1",
    "fid_period_div_code": "D"
}

In [28]:
res = requests.get(URL, headers=headers, params=params)

In [29]:
len(res.json()["output"])

30

In [30]:
res.json()["output"][0]

{'stck_bsop_date': '20220824',
 'stck_oprc': '59200',
 'stck_hgpr': '59500',
 'stck_lwpr': '59000',
 'stck_clpr': '59000',
 'acml_vol': '8888486',
 'prdy_vrss_vol_rate': '-1.69',
 'prdy_vrss': '-100',
 'prdy_vrss_sign': '5',
 'prdy_ctrt': '-0.17',
 'hts_frgn_ehrt': '49.80',
 'frgn_ntby_qty': '283206',
 'flng_cls_code': '00',
 'acml_prtt_rate': '1.00'}

## 3. 해쉬키 발급
### 보안을 위한 요소로 사용자가 보낸 요청 값을 중간에 탈취하여 변조하지 못하도록 하는데 사용, POST로 보내는 요청(주문)

In [31]:
datas = {
    "CANO": "00000000",
    "ACNT_PRDT_CD": "01",
    "OVRS_EXCG_CD": "SHAA",
    "PDNO": "00001",
    "ORD_QTY": "500",
    "OVRS_ORD_UNPR": "52.62",
    "ORD_SVR_DVSN_CD": "0"
}

headers = {
    "content-type": "application/json",
    "appkey": APP_KEY,
    "appsecret": APP_SECRET
}

PATH = "uapi/hashkey"
URL = f"{URL_BASE}/{PATH}"
print(URL)

https://openapivts.koreainvestment.com:29443/uapi/hashkey


In [32]:
res = requests.post(URL, headers=headers, data=json.dumps(datas))

In [33]:
hashkey = res.json()["HASH"]

## 4-1. 매수 주문
### tr_id : VTTC0802U

In [34]:
def hashKey(datas):
    PATH = "uapi/hashkey"
    URL = f"{URL_BASE}/{PATH}"
    headers = {
        'content-Type' : 'application/json',
        'appKey' : APP_KEY,
        'appSecret' : APP_SECRET,
        }
    res = requests.post(URL, headers=headers, data=json.dumps(datas))
    hashkey = res.json()["HASH"]

    return hashkey

In [35]:
PATH = "uapi/domestic-stock/v1/trading/order-cash"
URL = f"{URL_BASE}/{PATH}"

data = {
    "CANO": ACCOUNT_NUMBER1,
    "ACNT_PRDT_CD": ACCOUNT_NUMBER2,
    "PDNO": "005930", # 종목
    "ORD_DVSN": "01",
    "ORD_QTY": "10", # 수량
    "ORD_UNPR": "0", # 시장가 주문
    }

headers = {
    "content-type": "application/json", 
    "authorization": f"Bearer {ACCESS_TOKEN}",
    "appkey": APP_KEY,
    "appsecret": APP_SECRET,
    "tr_id": "VTTC0802U",
    "custtype": "P",
    "hashkey" : hashKey(data)
    }

In [36]:
res = requests.post(URL, headers=headers, data=json.dumps(data))
res.json()

{'rt_cd': '1', 'msg_cd': '40910000', 'msg1': '모의투자 주문이 불가한 계좌입니다.'}

In [37]:
KRX_FWDG_ORD_ORGNO = res.json()["output"]["KRX_FWDG_ORD_ORGNO"] # 한국거래소전송주문조직번호
ODNO = res.json()["output"]["ODNO"] # 주문번호

KeyError: 'output'

## 4-2. 정정 주문

In [ ]:
PATH = "uapi/domestic-stock/v1/trading/order-rvsencl"
URL = f"{URL_BASE}/{PATH}"

data ={
    "CANO": ACCOUNT_NUMBER1,
    "ACNT_PRDT_CD": ACCOUNT_NUMBER2,
    "KRX_FWDG_ORD_ORGNO": KRX_FWDG_ORD_ORGNO,
    "ORGN_ODNO": ODNO,
    "RVSE_CNCL_DVSN_CD": "02", # "01": 정정, "02": 취소
    "ORD_DVSN": "00",
    "ORD_QTY": "10",
    "ORD_UNPR": "0",
    "QTY_ALL_ORD_YN": "Y",
}